<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#load-data" data-toc-modified-id="load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>load data</a></span></li><li><span><a href="#Viz" data-toc-modified-id="Viz-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Viz</a></span></li><li><span><a href="#Features-selection" data-toc-modified-id="Features-selection-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Features selection</a></span><ul class="toc-item"><li><span><a href="#chi-squared-test" data-toc-modified-id="chi-squared-test-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>chi-squared test</a></span></li></ul></li><li><span><a href="#separate-data" data-toc-modified-id="separate-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>separate data</a></span></li><li><span><a href="#create-pipeline" data-toc-modified-id="create-pipeline-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>create pipeline</a></span></li><li><span><a href="#Build-a-model" data-toc-modified-id="Build-a-model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Build a model</a></span></li><li><span><a href="#Resultats" data-toc-modified-id="Resultats-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Resultats</a></span></li></ul></div>

In [1]:
#import all libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

#preprocessing and model selection
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedKFold, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline

#regression model metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error, make_scorer

#regression models
from sklearn.svm import LinearSVR

from sklearn import set_config
set_config(display='diagram')

from functions import *

## load data 

In [2]:
#Load data for 2019
path1 = "data/caracteristiques-2019.csv"
path2 = "data/lieux-2019.csv"
path3 = "data/usagers-2019.csv"
path4 = "data/vehicules-2019.csv"

In [3]:
pd.set_option('display.max_rows', None)

In [4]:
target = 'grav'
caracteristiques = pd.read_csv(path1, sep=';')
lieux = pd.read_csv(path2, sep=';')
usagers = pd.read_csv(path3, sep=';')
vehicules = pd.read_csv(path4, sep=';')

FileNotFoundError: [Errno 2] File data/caracteristiques-2019.csv does not exist: 'data/caracteristiques-2019.csv'

In [ ]:
def clean_caracteristiques(caracteristiques):

    # drop_caracteristiques = ['gps', 'adr', 'com'] # pour <--2018
    drop_caracteristiques = ['adr', 'com']  # pour 2019 -->


    new_values = \
['01', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
    '02', '21', '22', '23', '24', '25', '26', '27', '28', '29',
    '2A', '2B',
    '03', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39',
    '04', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49',
    '05', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59',
    '06', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69',
    '07', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79',
    '08', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
    '09', '90', '91', '92', '93', '94', '95',
    'QP', 'MQ', 'GF', 'RE', 'YT']

    old_values = \
['10', '100', '110', '120', '130', '140', '150', '160', '170', '180', '190',
    '20', '210', '220', '230', '240', '250', '260', '270', '280', '290',
    '201', '202',
    '30', '300', '310', '320', '330', '340', '350', '360', '370', '380', '390',
    '40', '400', '410', '420', '430', '440', '450', '460', '470', '480', '490',
    '50', '500', '510', '520', '530', '540', '550', '560', '570', '580', '590',
    '60', '600', '610', '620', '630', '640', '650', '660', '670', '680', '690',
    '70', '700', '710', '720', '730', '740', '750', '760', '770', '780', '790',
    '80', '800', '810', '820', '830', '840', '850', '860', '870', '880', '890',
    '90', '900', '910', '920', '930', '940', '950',
    '971', '972', '973', '974', '976']

    # drop columns
    caracteristiques = caracteristiques.drop(drop_caracteristiques, axis=1)
    # clean dep
    caracteristiques.dep = caracteristiques.dep.replace(old_values, new_values)
    # clean an
    caracteristiques.loc[(caracteristiques.an < 1000),
                         'an'] = caracteristiques.an + 2000
    caracteristiques.an = caracteristiques.an.astype('int')

    # hrmn
    for ind in caracteristiques[(caracteristiques.hrmn.str.len() < 3)].index:
        #print(len(caracteristiques[(caracteristiques.hrmn.str.len() < 3)].index))
        #print(ind)
        if caracteristiques.loc[ind, 'lum'] == 1:
            hm = str(choice([9, 10, 11, 12, 13, 14, 15, 16, 17
                             ])).zfill(2) + ':' + str(randint(0, 59)).zfill(2)
        if caracteristiques.loc[ind, 'lum'] == 2:
            hm = str(choice([6, 7, 19, 20])).zfill(2) + ':' + str(
                randint(0, 59)).zfill(2)
        if caracteristiques.loc[ind, 'lum'] in [3, 4, 5]:
            hm = str(choice([23, 0, 1, 2, 3, 4])).zfill(2) + ':' + str(
                randint(0, 59)).zfill(2)

        caracteristiques.loc[ind, 'hrmn'] = hm

    # separate numbers with : for time format
    caracteristiques.hrmn = [
        hm[:-2] + ':' + hm[-2:] if ':' not in hm else hm
        for hm in caracteristiques.hrmn
    ]

    ## change format str to time object
    caracteristiques.hrmn = [
        datetime.strptime(hm, '%H:%M').time()
        if type(hm) != type(datetime.now().time()) else hm
        for hm in caracteristiques.hrmn
    ]

    caracteristiques['hour'] = [hm.hour for hm in caracteristiques.hrmn]

    caracteristiques.drop(['hrmn'], axis=1, inplace=True)

    # weekday new columns
    caracteristiques['weekday'] = [
        datetime(an, mois, jour).weekday() for an, mois, jour in zip(
            caracteristiques.an, caracteristiques.mois, caracteristiques.jour)
    ]

    #lat
    caracteristiques['lat'] = caracteristiques['lat'].str.replace(',', '.')
    caracteristiques.lat = caracteristiques.lat.astype(float)
    caracteristiques.loc[(caracteristiques.lat > 10000),
                         'lat'] = caracteristiques.lat / 100000

    #long
    caracteristiques['long'] = caracteristiques['long'].str.replace(',', '.')
    caracteristiques['long'] = [
        np.nan if l == '-' else l for l in caracteristiques['long']
    ]
    caracteristiques['long'] = caracteristiques['long'].astype(float)
    caracteristiques.loc[(caracteristiques['an'] != 2019),
                         'long'] = caracteristiques['long'] / 100000

    #atm
    caracteristiques.atm = caracteristiques.atm.replace([np.nan, -1], 1)

    return (caracteristiques)


def clean_lieux(lieux):

    # do cleaning here
    #lieux = lieux.drop(['v1','v2','vma', 'env1'], axis = 1) # <---2018
    lieux = lieux.drop(['v1', 'v2'], axis=1)  # 2019 -->

    lieux.catr = lieux.catr.replace(np.nan, 4)

    lieux.circ = lieux.circ.replace([np.nan, -1], 0)

    lieux.nbv = lieux.nbv.replace([-1., np.nan], 0.)

    ## Assumtion 1
    ## Important assumtion cut all accidents where number of roads >10 or negative! Delete around 800 cases
    lieux = lieux[(lieux.nbv <= 10) | (lieux.nbv < 0)]

    lieux.vosp = lieux.vosp.replace([-1., np.nan], 0.)
    lieux.vosp = lieux.vosp.astype(int)

    lieux.prof = lieux.prof.replace([np.nan, -1.], 1.)

    lieux.pr = lieux.pr.replace([np.nan, '(1)', ''], 0.)
    lieux.pr1 = lieux.pr1.replace([np.nan, '(1)', ''], 0.)

    lieux.plan = lieux.plan.replace([-1., np.nan], 0.)

    lieux.surf = lieux.surf.replace([np.nan, -1., 0.])

    lieux.infra = lieux.infra.replace([np.nan, -1.], 0.)
    lieux.situ = lieux.situ.replace([np.nan, -1], 1)

    ## Assumtion 2
    #assumtion
    lieux.lartpc = lieux.lartpc.replace(np.nan, 0.)
    lieux.larrout = lieux.larrout.replace(np.nan, 0.)
    
     ## Assumtion 3
    #assumtion cut all accidents with the speed higher that 130 km/h
    lieux = lieux[(lieux.vma>0) & (lieux.vma<130)]
  

    return (lieux)

In [ ]:
def clean_usagers(usagers):
    usagers = usagers.drop(columns="id_vehicule")
    # On gère les valeurs dans les colonnes secu, pour qu'elles ressemblent à celle de la colonne secu

    # <--- 2018
    # usagers['secu'] = usagers['secu'].apply(lambda x: float(str(x)[1]) if x >= 10 else x )
    # usagers['secu']= usagers['secu'].replace(0,usagers['secu1'].median())

    usagers['secu1'] = usagers['secu1'].apply(lambda x: 3 if x == 8 else x)
    usagers['secu1'] = usagers['secu1'].apply(lambda x: 2 if x == 0 else x)
    usagers['secu1'] = usagers['secu1'].apply(
        lambda x: 1 if x != 3 and x != 2 and x != -1 else x)
    usagers['secu1'] = usagers['secu1'].replace(-1.0,
                                                usagers['secu1'].median())

    # On combine les colonnes puis on drop les colonnes secu 1,2,3
    # usagers['secu'] = usagers['secu'].fillna(usagers['secu1'])
    usagers = usagers.drop(columns=["secu1", "secu2", "secu3"])

    # Pour la colonne place, si la place n'est pas renseigné, on prend la mediane
    usagers['place'] = usagers['place'].fillna(usagers['place'].median())

    #localisation pieton locp

    usagers['locp'] = usagers['locp'].replace(-1.0, np.nan)
    usagers['locp'] = usagers['locp'].replace(9, np.nan)

    # Si dans la colonne catu il s'agit d'un piéton ( valeur 3) on remplace les Nan et -1 par les même proportion
    cond = (usagers['catu'] == 3)
    usagers.loc[cond, 'locp'] = usagers.loc[cond, 'locp'].fillna(
        pd.Series(
            np.random.choice([5.0, 1.0, 4.0, 6.0, 7.0, 8.0, 3.0, 2.0],
                             p=[
                                 0.068641, 0.157514, 0.260471, 0.022509,
                                 0.001271, 0.011444, 0.222107, 0.256043
                             ],
                             size=len(usagers))))

    # de fait, les Nan restant sont forcément autres que des piétons

    usagers['locp'] = usagers["locp"].fillna(0)

    locp = usagers[usagers.locp != 0]
    locp.locp.value_counts(normalize=True)

    # trajet, on comble les NaN pour garder les mêmes proportions
    usagers['trajet'] = usagers['trajet'].replace(-1.0, np.nan)
    usagers['trajet'] = usagers['trajet'].replace(0, np.nan)

    usagers['trajet'].value_counts(normalize=True)
    usagers['trajet'] = usagers['trajet'].fillna(
        pd.Series(
            np.random.choice([5.0, 1.0, 4.0, 9.0, 3.0, 2.0],
                             p=[0.52, 0.18, 0.14, 0.1, 0.03, 0.03],
                             size=len(usagers))))

    #actp même méthode que pour locp

    usagers['actp'] = usagers['actp'].replace([' -1', '7', '8', 'A', 'B'],
                                              np.nan)

    # Si dans la colonne catu il s'agit d'un piéton ( valeur 3) on remplace les Nan et -1 par les même proportion
    cond = (usagers['catu'] == 3)
    usagers.loc[cond, 'actp'] = usagers.loc[cond, 'actp'].fillna(
        pd.Series(
            np.random.choice([3.0, 9.0, 5.0, 1.0, 2.0, 4.0, 6.0],
                             p=[
                                 0.763227, 0.070539, 0.059319, 0.057122,
                                 0.028222, 0.018598, 0.002973
                             ],
                             size=len(usagers))))

    # de fait, les Nan restant sont forcément autres que des piétons

    usagers['actp'] = usagers['actp'].fillna(0)
    usagers['actp'] = usagers['actp'].astype(int)

    #etap piéton seul ou non, peu d'info, on drop
    usagers = usagers.drop(columns='etatp')

    #année de naissance, on remplace les inconnus par la valeur médiane
    usagers['an_nais'] = usagers['an_nais'].fillna(usagers['an_nais'].median())

    return usagers

In [ ]:
def clean_vehicules(vehicules):
    vehicules.drop(columns=['id_vehicule', 'motor','senc'], inplace = True)
    vehicules.choc = vehicules.choc.replace(-1,0)
    vehicules.obs = vehicules.obs.replace(-1,0)
    vehicules.obsm = vehicules.obsm.replace(-1,0)
    vehicules.manv = vehicules.manv.replace(-1,0) 
    return vehicules

#['obs', 'obsm', 'choc', 'manv', 'vma', 'col', 'lat', 'long']

In [ ]:
caracteristiques = clean_caracteristiques(caracteristiques)
lieux = clean_lieux(lieux)
vehicules = clean_vehicules(vehicules)
usagers = clean_usagers(usagers)

In [ ]:
caracteristiques_lieux = lieux.join(caracteristiques.set_index('Num_Acc'), on = 'Num_Acc')
vehicules_usagers = pd.merge(usagers, vehicules, on = ['Num_Acc','num_veh'])
df = vehicules_usagers.join(caracteristiques_lieux.set_index('Num_Acc'), on = 'Num_Acc')

df.head()

In [ ]:
df = df.dropna(subset=['mois'], axis = 0)

In [ ]:
info = df_info(df)

## Viz

In [ ]:
df_viz = df.drop(['Num_Acc', 'num_veh', 'occutc', 'voie', 'long', 'lat'], axis=1)

labels = ["Indemne", "Tué", "Blessé hospitalisé", "Blessé léger"]
# Create new dataframe column with the labels instead of numbers
df_viz["grav"] = df_viz["grav"].map(dict(zip(range(1, 5), labels)))

In [ ]:
df_viz = df_viz.dropna(axis = 0)

In [ ]:
for col in df_viz.columns:
    if df_viz[col].dtypes == 'float' :
        df_viz[col] = df_viz[col].astype('int')

In [ ]:
info_clean = df_info(df_viz)

In [ ]:
sns.histplot(data = df_viz[df_viz['catv']<40], x = 'catv', hue = 'grav', binwidth=0.99, palette = 'tab10',multiple="stack")

In [ ]:
sns.histplot(data = df_viz, x = 'catu', hue = 'grav', binwidth=0.999, palette = 'tab10', multiple="stack")

In [ ]:
sns.histplot(data = df_viz, x = df_viz.hour, hue = 'grav', binwidth=0.99, palette = 'tab10', multiple="stack")

In [ ]:
sns.histplot(data = df_viz, x = df_viz.weekday, hue = 'grav', binwidth=0.99, palette = 'tab10', multiple="stack")

In [ ]:
sns.histplot(data = df_viz, x = df_viz.mois, hue = 'grav', binwidth=0.99, palette = 'tab10', multiple="stack")

In [ ]:
sns.histplot(data = df_viz, x = df_viz.jour, hue = 'grav', binwidth=0.99, palette = 'tab10', multiple="stack")

In [ ]:
sns.histplot(data = df_viz, x = df_viz.atm, hue = 'grav', binwidth=0.25, palette = 'tab10', multiple="stack")

In [ ]:
sns.histplot(data = df_viz, x = df_viz.lum, hue = 'grav', binwidth=0.25, palette = 'tab10', multiple="stack")

In [ ]:
sns.histplot(data = df_viz, x = df_viz.catu, hue = 'grav', binwidth=0.99, palette = 'tab10', multiple="stack")

In [ ]:
sns.histplot(data = df_viz, x = df_viz.manv, hue = 'grav', binwidth=0.99, palette = 'tab10', multiple="stack")

In [ ]:
display(info_clean[info_clean['type']=='object'])


drop_for_model = ['an', 'mois', 'jour', 'Num_Acc', 'num_veh']



In [ ]:
drop = ['Num_Acc', 'num_veh']
df.drop(columns=drop, inplace = True)

In [ ]:
delete_null_cols = [col for col in info[info['null%']>85]['name']]
#those are ower null columns
delete_null_cols  #takeaway

##  Features selection

In [ ]:
info = df_info(df)

### chi-squared test

In [ ]:
df.an_nais.unique()

In [ ]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2
#separate target from features
X = df_model[['catu', 'lum', 'place', 'manv', 'choc', 'obsm', 'vma']]#.drop(columns = [target])
y = df_model[target]
print(X.shape)

ohc = OneHotEncoder(handle_unknown='ignore').fit(X)
X_trans = ohc.transform(X)
X_new = SelectPercentile(chi2, percentile=20).fit_transform(X_trans, y)
X_new.shape
print(X_new.shape)

## separate data

In [ ]:
# df_model = df.drop(['Num_Acc', 'num_veh', 'occutc', 'voie', 'long', 'lat', 'dep', 'an', 'mois', 'jour'], axis=1)
# df_model = df_model.dropna(axis = 0)

# numerical_features = ['pr', 'pr1', 'lartpc', 'larrout', 'an_nais']
# for col in df_model.columns:
#     if col not in numerical_features:
#         if df_model[col].dtypes == 'float':
#             df_model[col] = df_model[col].astype('int')


In [ ]:
#separate target from features
df_model = df
df_model['pr'] = df_model['pr'].astype('float')
df_model['pr1'] = df_model['pr1'].astype('float')

X = df_model.drop(columns = [target])
y = df_model[target]

In [ ]:
negative_cols = []
for col in df_model.columns:
    if df_model[col].dtypes != 'object':
        if len(df[df_model[col]<0])>0:
            print("numeric: {}".format(col))
            negative_cols.append(col)
            
        
    else:
        
        print("object:{}".format(col))
    

In [ ]:
df_model.head()

In [ ]:
info_model = df_info(df_model)

In [ ]:
# pieton_drop = ['locp', 'actp', 'etatp']
# pietons = df_model[df_model['catu'].isin([3,4])]
# usagers_en_voiture = df_model[df_model['catu'].isin([1,2])]
# usager.head()



In [ ]:
df_model.vma.value_counts()

## create pipeline

In [ ]:
# 1 drop
drop_features = ['Num_Acc', 'num_veh', 'occutc', 'voie', 'long', 'lat', 'dep', 'an', 'mois', 'jour']

# 2 numerical without log transformation
numerical_features = ['pr', 'pr1', 'lartpc', 'larrout', 'an_nais', 'vma']

numerical_scale_features = [
    feature for feature in numerical_features
    if feature not in drop_features
]

#3 categorical - dummies
categorical_features = [feature
    for feature in df.columns 
    if (feature not in drop_features and feature not in numerical_scale_features)
]
categorical_features.remove('grav')

print(drop_features)
print(categorical_features)
print(numerical_scale_features)

In [ ]:
#create Pipeline with transformation + model
import numpy as np
from sklearn.preprocessing import FunctionTransformer

numeric_scale_transformer = Pipeline(
    steps=[('imputer1', SimpleImputer(strategy='most_frequent')), 
           ('imputer2', SimpleImputer(missing_values = -1., strategy='most_frequent')),
           ('scaler', MinMaxScaler()) #StandardScaler() RobustScaler()), 
          ])

categorical_transformer = Pipeline(steps = [
    ('imputer1', SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')),
    ('imputer2', SimpleImputer(missing_values = -1, strategy = 'most_frequent')), 
    ('imputer3', SimpleImputer(strategy = 'most_frequent')), 
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[
    ('drop', 'drop', drop_features),
    ('num_scal', numeric_scale_transformer, numerical_scale_features),
    ('cat', categorical_transformer, categorical_features)
])


In [ ]:
preprocessor.fit(X)
X1 = preprocessor.transform(X)
X1

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2


from sklearn.ensemble import AdaBoostRegressor
#AdaBoostRegressor(base_estimator=None, *, n_estimators=50, learning_rate=1.0, loss='linear', random_state=None)

pipeline = Pipeline(steps=[('preprocessor',preprocessor),
                           ('feature_selection', SelectPercentile(chi2, percentile=30)), 
                           ('classifier', AdaBoostClassifier(base_estimator = SVC(max_iter = 5000, probability = True), n_estimators=20)
)])
pipeline

In [ ]:
# model = TransformedTargetRegressor(regressor=pipeline, transformer= target_transform)
# model
model = pipeline
model.get_params()

## Build a model

In [ ]:
#split data set on test and train data
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, #y.ravel(),
                                                    test_size=0.20,
                                                    random_state=19)

In [ ]:
params = \
{'classifier__algorithm': ['SAMME.R'],
 #'classifier__base_estimator__ccp_alpha': 0.0,
 #'classifier__base_estimator__class_weight': [None],
 #'classifier__base_estimator__criterion': 'gini',
 #'classifier__base_estimator__max_depth': [8, 10, 12],
 'classifier__base_estimator__max_features': [15, 30, 45],
 'classifier__base_estimator': DecisionTreeClassifier(),
 'classifier__learning_rate': [1.0, 5.],
 'classifier__n_estimators': [50, 100, 150]
}


params = \
{'classifier__algorithm': ['SAMME.R'],
 'classifier__base_estimator__max_iter': [5000],
 'classifier__learning_rate': [1.0, 5.],
 'classifier__n_estimators': [50, 100, 150]
}


In [ ]:
#cross validation
cv1 = RepeatedKFold(n_splits=4, n_repeats=1, random_state=19)
# #params
# Grid = GridSearchCV(model, params, scoring='r2', cv=cv1, n_jobs =-1) #root_mean_square_log_error   //n_iter = 10, 
# Grid.fit(X_train, y_train) #(n,1) -> (n,)
# best_model = Grid.best_estimator_

from sklearn.model_selection import cross_val_score
cross_val_score(model, X_train, y_train, scoring = 'f1_macro', cv=cv1, error_score="raise", n_jobs=3)

In [ ]:
Grid.cv_results_

In [ ]:
#check the parameters of the best model
best_model.get_params

## Resultats

In [ ]:
show_classification_model_metrics(model, X_test, y_test)